# Enhancing RAG with Contextual Retrieval

This notebook demonstrates contextual retrieval techniques using a modular, clean codebase.

## Overview
- **Basic RAG**: Standard document chunking and embedding
- **Contextual RAG**: Enhanced embeddings with situational context
- **Combined RAG**: Multi-document search with source attribution


In [1]:
# Import modules
import json
from vector_db import VectorDB, ContextualVectorDB
from rag_operations import (
    retrieve_base, retrieve_contextual, retrieve_combined,
    answer_query_base, answer_query_contextual, answer_query_combined
)
from data_utils import (
    transform_data_for_vectordb, create_contextual_dataset, 
    create_combined_dataset, add_contextual_information
)
import config


## 1. Basic RAG Setup


In [2]:
# Load and transform employee handbook data
with open('../data/employee_handbook.json', 'r') as f:
    employee_handbook_raw = json.load(f)

# Transform data for VectorDB
employee_handbook = transform_data_for_vectordb(employee_handbook_raw, "employee_handbook")

# Initialize and load VectorDB
db = VectorDB("employee_handbook")
db.load_data(employee_handbook)

print(f"Loaded {len(employee_handbook[0]['chunks'])} chunks into VectorDB")


Loading vector database from disk.
Loaded 77 chunks into VectorDB


In [13]:
# Test basic RAG system
test_query = "Do we get any sort of wfh-setup-refresh benefits?"
answer = answer_query_base(test_query, db)
print(f"Query: {test_query}")
print(f"Answer: {answer}")


Query: Do we get any sort of wfh-setup-refresh benefits?
Answer: Yes, Uniswap Labs provides a Home Office Set up benefit for remote team members. The company reimburses up to $2,000 USD to cover the purchase of office supplies, productivity items, and anything else you might need to get your home office set up.

Additionally, if you prefer to work from a co-working space, Uniswap Labs reimburses the cost up to $500 USD per month for co-working space expenses.

The company also provides a $50 per month cell phone stipend for benefits-eligible team members that is automatically added to your paychecks. This covers business-related use of personal cell phones, including both services and costs related to their devices. The stipend is non-taxable and is payable for every full month of employment.

Beyond the initial home office setup, Uniswap Labs provides you with a company issued computer to do your job.

Double-check with Julian or Megan for any of this information!


## Contextual RAG, Creating JSON

In [4]:
# Create contextual JSON for Employee Handbook
import json
from anthropic import Anthropic
from tqdm import tqdm

# Initialize Anthropic client
client = Anthropic()

# Template for the full employee handbook document
HANDBOOK_DOCUMENT_CONTEXT_PROMPT = """
<document>
This is Uniswap Labs' comprehensive Employee Handbook for team members. 
It covers company policies, employment guidelines, code of conduct, workplace policies, 
compensation, benefits, and operational procedures.

{doc_content}
</document>
"""

# Template for contextualizing individual chunks
HANDBOOK_CHUNK_CONTEXT_PROMPT = """
Here is a specific section from Uniswap's Employee Handbook that we want to situate within the overall handbook:

<chunk>
Section: {chunk_heading}
{chunk_content}
</chunk>

Please provide a short, succinct context to situate this handbook section within Uniswap's overall employee policies and procedures. This context will help employees find this information when searching for related policy topics.

Answer only with the succinct context and nothing else.
"""

def situate_handbook_context(full_handbook_doc: str, chunk_heading: str, chunk_content: str) -> str:
    """
    Generate contextual information for a handbook chunk within the full handbook document
    """
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=150,  # Keep context concise
        temperature=0.0,
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": HANDBOOK_DOCUMENT_CONTEXT_PROMPT.format(doc_content=full_handbook_doc),
                        "cache_control": {"type": "ephemeral"}  # Cache the full handbook doc
                    },
                    {
                        "type": "text",
                        "text": HANDBOOK_CHUNK_CONTEXT_PROMPT.format(
                            chunk_heading=chunk_heading,
                            chunk_content=chunk_content
                        ),
                    }
                ]
            }
        ],
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
    )
    return response.content[0].text.strip()

def add_contextual_information_handbook(input_file, output_file, original_markdown_file):
    """
    Process all employee handbook chunks to add contextual information
    """
    # Load the chunked JSON data
    with open(input_file, 'r') as f:
        chunks = json.load(f)
    
    # Load the original full markdown document for context
    with open(original_markdown_file, 'r') as f:
        full_handbook_doc = f.read()
    
    print(f"Processing {len(chunks)} employee handbook chunks...")
    
    enhanced_chunks = []
    
    for chunk in tqdm(chunks, desc="Adding contextual information"):
        try:
            # Generate situational context
            situational_context = situate_handbook_context(
                full_handbook_doc=full_handbook_doc,
                chunk_heading=chunk['chunk_heading'],
                chunk_content=chunk['text']
            )
            
            # Create enhanced chunk
            enhanced_chunk = {
                "chunk_link": chunk["chunk_link"],
                "chunk_heading": chunk["chunk_heading"],
                "text": chunk["text"],
                "situational_context": situational_context
            }
            
            enhanced_chunks.append(enhanced_chunk)
            
        except Exception as e:
            print(f"Error processing chunk '{chunk['chunk_heading']}': {e}")
            # Add chunk without context if there's an error
            enhanced_chunks.append(chunk)
    
    # Save the enhanced chunks
    with open(output_file, 'w') as f:
        json.dump(enhanced_chunks, f, indent=2)
    
    print(f"Contextual information added! Enhanced chunks saved to {output_file}")
    return enhanced_chunks

print("Functions defined for creating contextual employee handbook JSON!")


Functions defined for creating contextual employee handbook JSON!


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

input_file = 'data/employee_handbook.json'
output_file = 'data/employee_handbook_with_context.json'
original_markdown_file = '../../raw_data_and_data_cleaning/md files/Employee Handbook.md'

def add_contextual_information_handbook(input_file, output_file, original_markdown_file, max_workers=1):
    """
    Process all employee handbook chunks to add contextual information in parallel
    """
    # Load the chunked JSON data
    with open(input_file, 'r') as f:
        chunks = json.load(f)
    
    # Load the original full markdown document for context
    with open(original_markdown_file, 'r') as f:
        full_handbook_doc = f.read()
    
    print(f"Processing {len(chunks)} employee handbook chunks with {max_workers} workers...")
    
    def process_single_chunk(chunk):
        try:
            # Generate situational context
            situational_context = situate_handbook_context(
                full_handbook_doc=full_handbook_doc,
                chunk_heading=chunk['chunk_heading'],
                chunk_content=chunk['text']
            )

            # Print the chunk info
            print(f"\n{'='*60}")
            print(f"Chunk: {chunk['chunk_heading']}")
            print(f"Situational Context: {situational_context}")
            print(f"{'='*60}")
            
            # Create enhanced chunk
            return {
                "chunk_link": chunk["chunk_link"],
                "chunk_heading": chunk["chunk_heading"],
                "text": chunk["text"],
                "situational_context": situational_context
            }
        except Exception as e:
            print(f"Error processing chunk '{chunk['chunk_heading']}': {e}")
            return chunk
    
    enhanced_chunks = []
    
    # Process chunks in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all chunks for processing
        future_to_chunk = {executor.submit(process_single_chunk, chunk): chunk for chunk in chunks}
        
        # Collect results as they complete
        for future in tqdm(as_completed(future_to_chunk), total=len(chunks), desc="Adding contextual information"):
            result = future.result()
            enhanced_chunks.append(result)
    
    # Save the enhanced chunks
    with open(output_file, 'w') as f:
        json.dump(enhanced_chunks, f, indent=2)
    
    print(f"Contextual information added! Enhanced chunks saved to {output_file}")
    return enhanced_chunks


enhanced_chunks = add_contextual_information_handbook(
    input_file=input_file,
    output_file=output_file,
    original_markdown_file=original_markdown_file
)

Processing 77 employee handbook chunks with 1 workers...


Adding contextual information:   1%|▏         | 1/77 [00:02<02:34,  2.03s/it]


Chunk: For our team members working outside of New York State:
Situational Context: This section provides important information for Uniswap Labs employees working outside of New York State. It outlines that the handbook covers policies applicable to all Uniswap employees, but also includes specific information for New York-based employees. It clarifies that employees in other states may be subject to different state-specific policies and benefits, and may receive supplemental state-specific handbooks. This section also reiterates Uniswap's at-will employment policy, which applies to all employees regardless of location.


Adding contextual information:   3%|▎         | 2/77 [00:04<02:31,  2.02s/it]


Chunk: **Uniswap Principles | Uni-code**
Situational Context: This section on "Uniswap Principles | Uni-code" outlines Uniswap's core values and guiding principles that shape the company's culture and how employees interact with each other, users, and the broader community. These principles serve as daily guideposts for Uniswap's performance-driven and collaborative work environment, and are foundational to the company's mission of building a people-first, internet-native financial system.


...

Adding contextual information:   9%|▉         | 7/77 [00:12<01:59,  1.71s/it]


Chunk: 
Situational Context: This section on Compliance with Anti-Corruption and Anti-Money Laundering Laws is part of Uniswap Labs' overall policies and procedures that outline the company's commitment to conducting business ethically and in full compliance with all applicable laws and regulations. It provides guidance to employees on the company's expectations and requirements around anti-corruption and anti-money laundering practices.


Adding contextual information:  10%|█         | 8/77 [00:14<02:04,  1.80s/it]


Chunk: Social Media Guidelines  {#social-media-guidelines}
Situational Context: This section on Social Media Guidelines is part of Uniswap's overall employee handbook, which covers company policies, employment guidelines, code of conduct, workplace policies, compensation, benefits, and operational procedures. It provides guidance to Uniswap employees on how to responsibly engage with the company's social media platforms and their own personal accounts, with a focus on maintaining the company's reputation and brand.



Chunk: 
Situational Context: This section on Compliance with Anti-Corruption and Anti-Money Laundering Laws is part of Uniswap Labs' overall policies and procedures outlined in the Employee Handbook. It provides guidance to employees on Uniswap's commitment to conducting business ethically and in full compliance with applicable laws, including the Foreign Corrupt Practices Act (FCPA) and anti-money laundering regulations. This policy is critical to Uniswap's continued success and reputation as a responsible corporate citizen.

Chunk: Travel and Event Safety {#travel-and-event-safety}
Situational Context: This section on Travel and Event Safety is part of the "How We Keep You Safe" section of Uniswap's Employee Handbook. It outlines the company's policies and procedures for protecting the physical safety of employees when they participate in travel or attend events outside of the Uniswap office.


In [5]:
# Transform contextual JSON to VectorDB format and create pickle file
import json
import os
from vector_db import ContextualVectorDB

def transform_contextual_data_for_vectordb(contextual_data: list, doc_id: str = "employee_handbook_contextual") -> list:
    """Transform contextual JSON data to match ContextualVectorDB structure."""
    
    # Reconstruct the full document content for the ContextualVectorDB
    full_content = "\n\n".join([chunk["text"] for chunk in contextual_data])
    
    doc = {
        "doc_id": doc_id,
        "original_uuid": f"{doc_id}_doc",
        "content": full_content,  # Full document content for context generation
        "chunks": []
    }
    
    for i, item in enumerate(contextual_data):
        chunk = {
            "chunk_id": f"chunk_{i}",
            "original_index": i,
            "content": item["text"],  # Original content
            "heading": item["chunk_heading"],
            "link": item["chunk_link"],
            "contextual_content": item.get("situational_context", "")  # Pre-computed contextual info
        }
        doc["chunks"].append(chunk)
    
    return [doc]

# Load the contextual JSON data
print("📖 Loading contextual employee handbook data...")
with open('data/employee_handbook_with_context.json', 'r') as f:
    contextual_data = json.load(f)

print(f"✅ Loaded {len(contextual_data)} contextual chunks")

# Transform data for ContextualVectorDB
print("🔄 Transforming data for ContextualVectorDB...")
transformed_data = transform_contextual_data_for_vectordb(contextual_data)

print(f"📊 Transformed data structure:")
print(f"  - Document ID: {transformed_data[0]['doc_id']}")
print(f"  - Total chunks: {len(transformed_data[0]['chunks'])}")
print(f"  - Full content length: {len(transformed_data[0]['content'])} characters")


📖 Loading contextual employee handbook data...
✅ Loaded 77 contextual chunks
🔄 Transforming data for ContextualVectorDB...
📊 Transformed data structure:
  - Document ID: employee_handbook_contextual
  - Total chunks: 77
  - Full content length: 152067 characters


In [6]:
# Create a specialized ContextualVectorDB that uses pre-computed contextual information
from vector_db import ContextualVectorDB
import numpy as np
from tqdm import tqdm
from openai import OpenAI
import pickle
import config

class PreComputedContextualVectorDB(ContextualVectorDB):
    """Contextual Vector DB that uses pre-computed contextual information instead of generating it."""
    
    def __init__(self, name: str, openai_api_key: str = None):
        # Only initialize OpenAI client, skip Anthropic since we have pre-computed context
        if openai_api_key is None:
            openai_api_key = config.OPENAI_API_KEY
        
        self.openai_client = OpenAI(api_key=openai_api_key)
        self.name = name
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = f"./data/{name}/contextual_vector_db.pkl"

    def load_data_with_precomputed_context(self, dataset: list):
        """Load data with pre-computed contextual information."""
        if self.embeddings and self.metadata:
            print("Vector database is already loaded. Skipping data loading.")
            return
        if os.path.exists(self.db_path):
            print("Loading vector database from disk.")
            self.load_db()
            return

        texts_to_embed = []
        metadata = []
        total_chunks = sum(len(doc['chunks']) for doc in dataset)
        
        print(f"Processing {total_chunks} chunks with pre-computed contextual information...")
        
        with tqdm(total=total_chunks, desc="Processing chunks") as pbar:
            for doc in dataset:
                for chunk in doc['chunks']:
                    # Combine original content with pre-computed contextual information
                    contextual_text = f"{chunk['content']}\n\n{chunk['contextual_content']}"
                    
                    texts_to_embed.append(contextual_text)
                    metadata.append({
                        'doc_id': doc['doc_id'],
                        'original_uuid': doc['original_uuid'],
                        'chunk_id': chunk['chunk_id'],
                        'original_index': chunk['original_index'],
                        'original_content': chunk['content'],
                        'contextual_content': chunk['contextual_content'],
                        'heading': chunk.get('heading', ''),
                        'link': chunk.get('link', '')
                    })
                    pbar.update(1)

        self._embed_and_store(texts_to_embed, metadata)
        self.save_db()
        print(f"✅ Contextual Vector database loaded and saved. Total chunks processed: {len(texts_to_embed)}")

# Create the contextual vector database
print("🔧 Creating PreComputed Contextual Vector Database...")
contextual_db = PreComputedContextualVectorDB("employee_handbook_contextual")

# Load the transformed data
print("📊 Loading data into contextual vector database...")
contextual_db.load_data_with_precomputed_context(transformed_data)

print("🎉 Successfully created contextual vector database pickle file!")


🔧 Creating PreComputed Contextual Vector Database...
📊 Loading data into contextual vector database...
Loading vector database from disk.
🎉 Successfully created contextual vector database pickle file!


## 2. Contextual RAG Setup

We'll demonstrate contextual RAG using the Benefits & Wellbeing document. Contextual RAG enhances each chunk with situational context before embedding, leading to better retrieval accuracy.


In [7]:
# Example: Full RAG Pipeline - Retrieve + Generate Answer
from rag_operations import answer_query_contextual

# Load the contextual database
print("📖 Loading contextual employee handbook database...")
contextual_handbook_db = PreComputedContextualVectorDB("employee_handbook_contextual")
contextual_handbook_db.load_db()

📖 Loading contextual employee handbook database...


In [8]:
print("QUERY TO ANSWER")
print("=" * 60)

test_query = "What are Uniswap's core values?"

print(f"\nQuery: {test_query}")
print("-" * 50)

answer = answer_query_contextual(test_query, contextual_handbook_db)
print(f"\nAnswer: {answer}")

print(f"\n" + "="*60)
print("UNDERLYING RETRIEVAL DETAILS")
print("="*60)

# Also show what chunks were retrieved for transparency
results = contextual_handbook_db.search(test_query, k=3)
print(f"\n📊 Retrieved {len(results)} contextual chunks:")

for i, result in enumerate(results, 1):
    print(f"\n🏆 Chunk #{i} (Similarity: {result['similarity']:.4f})")
    print(f"📝 Section: {result['metadata'].get('heading', 'N/A')}")
    print(f"📄 Content: {result['metadata']['original_content'][:200]}...")
    print(f"🧠 Context: {result['metadata']['contextual_content']}")

print(f"\n✅ Contextual RAG pipeline completed successfully!")


QUERY TO ANSWER

Query: What are Uniswap's core values?
--------------------------------------------------

Answer: Uniswap's core values are articulated through their operating principles called "Unicode," which serve as daily guideposts for how they interact with each other, users, and their community.

**People First** emphasizes that Uniswap believes easy, safe, fair value transfer on the internet can improve people's lives, with access, security and experience at the center of everything they do. By pursuing decentralization, interoperability, and durability, they align with users over the long-term. Internally, people are their greatest asset, and they strive for an environment where everyone can make an incredible impact. They share direct, kind feedback for improvement and advocate for ideas by starting with why it's better for users and the company.

**Simple** reflects their craft of keeping things simple in a complex field by creating clarity and simplicity. They write and b

In [ ]:
# Example: Using the contextual vector database for RAG
from rag_operations import answer_query_contextual

# Load the contextual database
print("📖 Loading contextual employee handbook database...")
contextual_handbook_db = PreComputedContextualVectorDB("employee_handbook_contextual")
contextual_handbook_db.load_db()

# Test queries
test_query = [
    "What are Uniswap's core values?"
]

print("\n🔍 Testing RAG with contextual embeddings:")
print("-" * 50)

for i, query in enumerate(test_query, 1):
    print(f"\n{i}. Query: {query}")
    
    # Search for relevant chunks
    results = contextual_handbook_db.search(query, k=3)
    print(f"   📊 Found {len(results)} relevant chunks")
    print(f"   🎯 Top similarity score: {results[0]['similarity']:.4f}")
    print(f"   📝 Most relevant section: {results[0]['metadata'].get('heading', 'N/A')[:80]}...")


📖 Loading contextual employee handbook database...

🔍 Testing RAG with contextual embeddings:
--------------------------------------------------

1. Query: What are Uniswap's core values?
[{'metadata': {'doc_id': 'employee_handbook_contextual', 'original_uuid': 'employee_handbook_contextual_doc', 'chunk_id': 'chunk_1', 'original_index': 1, 'original_content': '## **Uniswap Principles | Uni-code**\n\n*Our Uniswap operating principles (Unicode) articulate who we are (our values) and how we work. They are our daily guideposts for how we interact with each other, our users, and our community.*\n\n**People First**\n\nWe believe easy, safe, fair value transfer on the internet can improve people’s lives. Access, security and experience is the center of everything we do. By pursuing decentralization, interoperability, and durability we align with our users over the long-term, and win. Internally, people are our greatest asset, and we strive for an environment where everyone can make an incredi

## Combined Contextual RAG (creating)

In [9]:
## Creating a Combined VectorDB with Multiple Documents

# Let's create a combined dataset that includes both benefits_wellbeing and employee_handbook
def create_combined_dataset():
    """
    Create a combined dataset with both benefits_wellbeing and employee_handbook documents
    """
    
    # Load both JSON files
    with open('data/benefits_wellbeing_with_context.json', 'r') as f:
        benefits_data_context = json.load(f)
    
    with open('data/employee_handbook_with_context.json', 'r') as f:
        handbook_data_context = json.load(f)
    
    print(f"Loaded {len(benefits_data_context)} benefits chunks")
    print(f"Loaded {len(handbook_data_context)} handbook chunks")
    
    def transform_to_combined_format(raw_data, doc_id, doc_type):
        """Transform data to VectorDB format with document identification"""
        doc = {
            "doc_id": doc_id,
            "original_uuid": f"{doc_id}_uuid",
            "doc_type": doc_type,  # Add document type for easier filtering
            "chunks": []
        }
        
        for i, item in enumerate(raw_data):
            chunk = {
                "chunk_id": f"{doc_id}_chunk_{i}",
                "original_index": i,
                "content": item["text"],
                "heading": item["chunk_heading"],
                "link": item["chunk_link"],
                "doc_type": doc_type,  # Also add to chunk metadata
                "source_doc": doc_id  # Clear source identification
            }
            doc["chunks"].append(chunk)
        
        return doc
    
    # Transform both datasets
    benefits_doc = transform_to_combined_format(benefits_data_context, "benefits_wellbeing", "benefits")
    handbook_doc = transform_to_combined_format(handbook_data_context, "employee_handbook", "handbook")
    
    # Combine into a single dataset
    combined_contextual_dataset = [benefits_doc, handbook_doc]
    
    # Save the combined dataset
    output_file = 'data/combined_documents_with_context.json'
    with open(output_file, 'w') as f:
        json.dump(combined_contextual_dataset, f, indent=2)
    
    total_chunks = len(benefits_doc["chunks"]) + len(handbook_doc["chunks"])
    print(f"✅ Created combined dataset with {total_chunks} total chunks")
    print(f"   - Benefits & Wellbeing: {len(benefits_doc['chunks'])} chunks")
    print(f"   - Employee Handbook: {len(handbook_doc['chunks'])} chunks")
    print(f"✅ Saved to: {output_file}")
    
    return combined_contextual_dataset

# Create the combined dataset
combined_contextual_dataset = create_combined_dataset()


Loaded 5 benefits chunks
Loaded 77 handbook chunks
✅ Created combined dataset with 82 total chunks
   - Benefits & Wellbeing: 5 chunks
   - Employee Handbook: 77 chunks
✅ Saved to: data/combined_documents_with_context.json


In [10]:
# Initialize a combined VectorDB
print("Creating combined VectorDB with both documents...")

# Initialize the VectorDB for combined documents
combined_db = VectorDB("combined_documents_with_context")
combined_db.load_data(combined_contextual_dataset)

total_chunks = sum(len(doc['chunks']) for doc in combined_contextual_dataset)
print(f"✅ Combined VectorDB ready! Total chunks: {total_chunks}")


Creating combined VectorDB with both documents...
Loading vector database from disk.
✅ Combined VectorDB ready! Total chunks: 82


In [11]:
# Enhanced RAG Functions for Combined Database

def retrieve_combined(query, combined_db, k=5):
    """Retrieve relevant documents from combined database with source information"""
    results = combined_db.search(query, k=k)
    context = ""
    sources = []
    
    for result in results:
        chunk = result['metadata']
        
        # Extract source information - handle the actual metadata structure
        doc_id = chunk.get('doc_id', 'unknown_doc')
        chunk_id = chunk.get('chunk_id', 'unknown_chunk')
        
        # Determine source document from doc_id
        if 'benefits' in doc_id.lower():
            source_doc = 'benefits_wellbeing'
        elif 'handbook' in doc_id.lower() or 'employee' in doc_id.lower():
            source_doc = 'employee_handbook'
        else:
            source_doc = doc_id
        
        # Try to extract heading from content (first line if it starts with #)
        content = chunk.get('content', '')
        heading = 'Unknown Section'
        content_lines = content.split('\n')
        for line in content_lines[:3]:  # Check first 3 lines
            if line.strip().startswith('#'):
                heading = line.strip().replace('#', '').strip()
                break
        
        # If no heading found, use chunk_id as fallback
        if heading == 'Unknown Section':
            heading = chunk_id
        
        # Include source information in context
        source_info = f"[Source: {source_doc} - {heading}]"
        context += f"\n{source_info}\n{content}\n"
        sources.append({
            'source_doc': source_doc,
            'heading': heading,
            'similarity': result['similarity']
        })
    
    return results, context, sources

def answer_query_combined(query, combined_db):
    """Answer a query using the Combined RAG pipeline"""
    documents, context, sources = retrieve_combined(query, combined_db)
    
    # Create source summary
    source_summary = "Sources consulted:\n"
    for source in sources:
        source_summary += f"• {source['source_doc']}: {source['heading']} (similarity: {source['similarity']:.3f})\n"
    
    prompt = f""" ### SYSTEM ###
    You are **Uniswap Employee Assistant**.

    You have been provided with relevant company documents from multiple sources to answer employee questions.

    **Workflow for this query:**
    1. **Analyze the user question**: {query}
    2. **Review the provided context** below for relevant information
    3. **Write a natural-language answer** following these rules:
    • Use only facts that appear verbatim in the provided context
    • If the information isn't in the context, reply: "I don't have that information in the provided context."
    • Compose the most **expansive, detailed answer possible** by weaving together **every relevant fact** found in the context—rephrasing, grouping, and elaborating on those facts for clarity and flow
    • You may explain terms, list related details, and provide a logical structure
    • **Never introduce information that is not stated verbatim in the context**
    • When referencing information, mention which document type it comes from (e.g., "According to the benefits documentation..." or "The employee handbook states...")
    4. **Do not** provide preamble such as "Here is the answer" or "Based on the documents"
    5. **Always append exactly**: "Double-check with Julian or Megan for any of this information!"

    ### USER QUESTION ###
    {query}

    ### CONTEXT ###
    {context}

    ### RESPONSE ###"""
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=500,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    
    # Handle the response content properly
    try:
        answer = response.content[0].text
    except AttributeError:
        answer = str(response.content[0])
    
    return answer, source_summary


## Combined Contextual RAG (Prompt)

In [12]:
# Test the combined RAG system with a query that might span both documents
test_query = "Tell me about the company's gym benefits"
answer, sources = answer_query_combined(test_query, combined_db)

print(f"Query: {test_query}")
print(f"\n{sources}")
print(f"Combined Answer: {answer}")

Query: Tell me about the company's gym benefits

Sources consulted:
• employee_handbook: Employee Benefits {employee-benefits} (similarity: 0.470)
• benefits_wellbeing: Health Benefits (similarity: 0.424)
• benefits_wellbeing: 401k & Financial Benefits (similarity: 0.393)
• employee_handbook: Phones (similarity: 0.356)
• employee_handbook: Reasonable Accommodations:  Disability, Nursing Mothers and Religious  {reasonable-accommodations:-disability,-nursing-mothers-and-religious} (similarity: 0.339)

Combined Answer: According to the provided company documents, I don't have information about specific gym benefits in the provided context. 

The employee handbook and benefits documentation cover various health and wellbeing benefits including medical coverage through Anthem Blue Cross and Kaiser, dental plans through Guardian, vision insurance through VSP, One Medical access, and Maven for fertility and family planning. The benefits documentation also mentions that "Uniswap offers a varie

# test

In [ ]:
# Force reload the module to pick up new functions
import importlib
import data_utils
importlib.reload(data_utils)

# Import the setup function from data_utils
from data_utils import setup_contextual_embeddings

# =====================================
# CHANGE DOCUMENT HERE - JUST ONE LINE!
# =====================================
DOCUMENT_NAME = 'employee_handbook'  # or 'employee_handbook' , 'benefits_wellbeing'

# Setup contextual embeddings
benefits_raw, enhanced_chunks = setup_contextual_embeddings(DOCUMENT_NAME)

# Create contextual dataset (includes full document content for context generation)
contextual_dataset = create_contextual_dataset()

print(f"Created contextual dataset with {len(contextual_dataset[0]['chunks'])} chunks")
print(f"Full document length: {len(contextual_dataset[0]['content'])} characters")


📖 Loading Employee Handbook data for contextual RAG...
🔄 Creating contextual embeddings for Employee Handbook...


FileNotFoundError: [Errno 2] No such file or directory: '../data/Employee Handbook.md'